In [2]:
!pip install tensorflow

^C


In [5]:
!python --version

Python 3.11.4
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e9

In [6]:
import tensorflow as tf
import numpy as np
import cv2
import tensorflow_hub as hub
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
from pycocotools.coco import COCO
import json
import random

ModuleNotFoundError: No module named 'cv2'

In [ ]:
ls -l ./annotations/

In [9]:
import json
import random
from pycocotools.coco import COCO

# Path to the annotations file
annotations_path = 'annotations/instances_train2017.json'

# Initialize COCO API for instance annotations
coco = COCO(annotations_path)

# Get all image ids
image_ids = coco.getImgIds()

# Modified annotations dictionary
modified_annotations = {'images': [], 'annotations': [], 'categories': coco.loadCats(coco.getCatIds())}

count = 0
# Process each image
for image_id in image_ids:
    # Load image information
    image_info = coco.loadImgs(image_id)[0]
    
    # Load annotations for this image
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)

    # Skip images with no annotations
    if not anns:
        continue

    # Randomly choose an annotation to delete
    index_to_remove = random.randint(0, len(anns) - 1)
    deleted_ann = anns.pop(index_to_remove)  # Remove the chosen annotation

    # Add image info and deleted object name to modified annotations
    modified_annotations['images'].append({
        'id': image_info['id'],
        'file_name': image_info['file_name'],
        'deleted_object': coco.loadCats([deleted_ann['category_id']])[0]['name']
    })

    # Add remaining annotations to the modified list
    modified_annotations['annotations'].extend(anns)

# Save the modified annotations to a new JSON file
with open('modified_instances_train2017.json', 'w') as f:
    json.dump(modified_annotations, f)

print("Modified annotations have been saved.")


loading annotations into memory...
Done (t=5.28s)
creating index...
index created!
Number of images in the original 'train2017' directory: 118287
Processing complete. Modified images and annotations are saved.


# Preparing the Model

In [11]:
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

num_steps = 50000
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    }
}

selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [39]:
%cd /home/johnchidiac/DeepLearning
!git clone https://github.com/tensorflow/models.git

/home/johnchidiac/DeepLearning
Cloning into 'models'...
remote: Enumerating objects: 97132, done.
remote: Counting objects: 100% (412/412), done.
remote: Compressing objects: 100% (199/199), done.
error: RPC failed; curl 92 HTTP/2 stream 0 was not closed cleanly: CANCEL (err 8)
error: 2183 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [41]:
%ls

 annotations/                  'Deep Learning Project.ipynb'   train2017/
 annotations_trainval2017.zip   modified_annotations/          val2017/
 content/                       modified_images/


In [42]:
%cd /home/johnchidiac/DeepLearning
!mkdir -p models/research/object_detection/dataset_tools
%cd models/research/object_detection/dataset_tools

/home/johnchidiac/DeepLearning
/home/johnchidiac/DeepLearning/models/research/object_detection/dataset_tools


In [43]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/dataset_tools/create_coco_tf_record.py

--2024-04-25 04:56:58--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/dataset_tools/create_coco_tf_record.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22961 (22K) [text/plain]
Saving to: ‘create_coco_tf_record.py’

create_coco_tf_reco 100%[===================>]  22.42K  --.-KB/s    in 0.1s    

2024-04-25 04:56:58 (208 KB/s) - ‘create_coco_tf_record.py’ saved [22961/22961]



In [47]:
%ls /home/johnchidiac/DeepLearning/output_tfrecords

ls: cannot access '/home/johnchidiac/DeepLearning/output_tfrecords': No such file or directory


In [48]:
!mkdir -p /home/johnchidiac/DeepLearning/output_tfrecords

# Script to generate TFRecords and it skips the missing files

In [53]:
import os
import hashlib
import io
import numpy as np
import PIL.Image
import tensorflow as tf

from pycocotools import mask
from object_detection.utils import dataset_util as du

def create_tf_example(image, annotations_list, image_dir, category_index, include_masks=False, include_keypoints=False):
    image_path = os.path.join(image_dir, image['file_name'])
    if not os.path.exists(image_path):
        print("Skipping missing file: {}".format(image_path))
        return None, 0, 0, 0

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()

    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = PIL.Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    width = int(image.width)
    height = int(image.height)

    xmin = []
    xmax = []
    ymin = []
    ymax = []
    is_crowd = []
    category_names = []
    category_ids = []
    area = []
    encoded_mask_png = []

    for object_annotations in annotations_list:
        (x, y, width, height) = tuple(object_annotations['bbox'])
        if width <= 0 or height <= 0:
            continue
        if x + width > image.width or y + height > image.height:
            continue
        xmin.append(float(x) / image.width)
        xmax.append(float(x + width) / image.width)
        ymin.append(float(y) / image.height)
        ymax.append(float(y + height) / image.height)
        is_crowd.append(object_annotations['iscrowd'])
        category_ids.append(int(object_annotations['category_id']))
        category_names.append(category_index[object_annotations['category_id']]['name'].encode('utf8'))
        area.append(object_annotations['area'])

        if include_masks:
            run_len_encoding = mask.frPyObjects(object_annotations['segmentation'], image.height, image.width)
            binary_mask = mask.decode(run_len_encoding)
            pil_image = PIL.Image.fromarray(binary_mask)
            output_io = io.BytesIO()
            pil_image.save(output_io, format='PNG')
            encoded_mask_png.append(output_io.getvalue())

    feature_dict = {
        'image/height': du.int64_feature(height),
        'image/width': du.int64_feature(width),
        'image/filename': du.bytes_feature(image['file_name'].encode('utf8')),
        'image/source_id': du.bytes_feature(image['file_name'].encode('utf8')),
        'image/key/sha256': du.bytes_feature(key.encode('utf8')),
        'image/encoded': du.bytes_feature(encoded_jpg),
        'image/format': du.bytes_feature('jpeg'.encode('utf8')),
        'image/object/bbox/xmin': du.float_list_feature(xmin),
        'image/object/bbox/xmax': du.float_list_feature(xmax),
        'image/object/bbox/ymin': du.float_list_feature(ymin),
        'image/object/bbox/ymax': du.float_list_feature(ymax),
        'image/object/class/text': du.bytes_list_feature(category_names),
        'image/object/class/label': du.int64_list_feature(category_ids),
        'image/object/is_crowd': du.int64_list_feature(is_crowd),
        'image/object/area': du.float_list_feature(area),
    }

    if include_masks:
        feature_dict['image/object/mask'] = du.bytes_list_feature(encoded_mask_png)

    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example, len(xmin), 0, 0


## Spliting the data into training and testing sets

In [73]:
%ls

coco_train.record-00000-of-00100  coco_train.record-00050-of-00100
coco_train.record-00001-of-00100  coco_train.record-00051-of-00100
coco_train.record-00002-of-00100  coco_train.record-00052-of-00100
coco_train.record-00003-of-00100  coco_train.record-00053-of-00100
coco_train.record-00004-of-00100  coco_train.record-00054-of-00100
coco_train.record-00005-of-00100  coco_train.record-00055-of-00100
coco_train.record-00006-of-00100  coco_train.record-00056-of-00100
coco_train.record-00007-of-00100  coco_train.record-00057-of-00100
coco_train.record-00008-of-00100  coco_train.record-00058-of-00100
coco_train.record-00009-of-00100  coco_train.record-00059-of-00100
coco_train.record-00010-of-00100  coco_train.record-00060-of-00100
coco_train.record-00011-of-00100  coco_train.record-00061-of-00100
coco_train.record-00012-of-00100  coco_train.record-00062-of-00100
coco_train.record-00013-of-00100  coco_train.record-00063-of-00100
coco_train.record-00014-of-00100  coco_train.record-00064-of-0

In [72]:
%cd ./output_tfrecords

/home/johnchidiac/DeepLearning/output_tfrecords


In [94]:
!cd ../..

In [95]:
!pwd

/home/johnchidiac/DeepLearning/output_tfrecords


In [97]:
import tensorflow as tf

# Define the path to your TFRecord file
tfrecord_file = '../output_tfrecords/coco_train.record-00000-of-00100'

# Define the function to parse TFRecord example
def parse_tfrecord_fn(example_proto):
    # Define your feature description dictionary
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/class/label': tf.io.FixedLenFeature([], tf.int64),
        # Add more features as needed
    }
    # Parse the example
    example = tf.io.parse_single_example(example_proto, feature_description)
    # Process and return the features
    return example['image/encoded'], example['image/object/class/label']

# Load and parse the TFRecord file
dataset = tf.data.TFRecordDataset(tfrecord_file)
dataset = dataset.map(parse_tfrecord_fn)

# Iterate over the dataset to print the first few examples
for image_encoded, label in dataset.take(5):  # Adjust the number taken as needed
    # Your processing logic here, e.g., decode image, convert label, etc.
    image = tf.io.decode_image(image_encoded)
    print("Image shape:", image.shape)
    print("Label:", label.numpy())


2024-04-25 05:57:08.445407: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: image/object/class/label (data type: int64) is required but could not be found.
2024-04-25 05:57:08.445572: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Feature: image/object/class/label (data type: int64) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: image/object/class/label (data type: int64) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

## Installing Required Packages

In [15]:
import os

%cd ./content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/home/johnchidiac/DeepLearning/content
Cloning into 'tensorflow-object-detection-faster-rcnn'...
remote: Enumerating objects: 885, done.
remote: Total 885 (delta 0), reused 0 (delta 0), pack-reused 885
Receiving objects: 100% (885/885), 24.83 MiB | 1.46 MiB/s, done.
Resolving deltas: 100% (428/428), done.
/home/johnchidiac/DeepLearning/content/tensorflow-object-detection-faster-rcnn
Already up to date.


In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

!python object_detection/builders/model_builder_test.py